In [1]:
import girder_client 
%load_ext autoreload
%autoreload 2
#import adrcSchemaHelpers as hlprs
import metadata_pipeline as mph 
import brain_region_maps as brm

#gc = girder_client.GirderClient(apiUrl="https://styx.neurology.emory.edu/girder/api/v1")
gc = girder_client.GirderClient(apiUrl="http://candygram.neurology.emory.edu:8080/api/v1")
_ = gc.authenticate(interactive=True)

Login or email: admin
Password for admin: ········


In [2]:
NeuroPathDemoImages_Collection='638e2da11f75016b81fda12f'

def getPtSlideDataSummary( slideSetMetaData,debug=True):
    ## Expects an array of all of the items below a folder, so for exampel  all of the slides for case E20-73
    ## This should then return the number of control slides found, number of files that are missing metadata
    ## completely, and also invalid metadata
    ctrlSlides = 0
    validatedSlides = 0
    notValid = 0
    slidesToReview = []
    
    for i in slideSetMetaData:
        if 'npSchema' not in i['meta']:
            notValid+=1
            slidesToReview.append(i)
        elif ('controlSlide' in i['meta']['npSchema']):
            ctrlSlides+=1
        else:
            validItem,meta = mph.validateMetadata(i['meta'])
            ### Now process errors from 
            if not validItem:
                notValid+=1
                slidesToReview.append(i)
            else:
                validatedSlides+=1
    if debug:
        pass
        #print("Valid:%d,Controls:%d,NotValid:%d" % (validatedSlides,ctrlSlides,notValid))
        
        
    return {"status":"parsed", "valid":validatedSlides,"controlSlides": ctrlSlides, "notValid": notValid, "slidesToReview":slidesToReview}

In [ ]:
# testOutput = mph.populateMetadata(gc, folderID='638e2dea1f75016b81fda132', outputFailed=True)


In [3]:
## Need to do this on a per case basis
needsBrainRegionMap = []
slideSummaryData = {}

for yrf in gc.listFolder(NeuroPathDemoImages_Collection,parentFolderType='collection'):
    year = yrf['name']
    if year.isdigit():
        slideSummaryData[year] = {} ## Going to create a stats dictionary ordered by year/pt
        
        for pt in gc.listFolder(yrf['_id']):
            slideSummaryData[year][pt['name']] = {}
            
            if pt['name'] not in brm.MAP:
                #print(year,pt['name'],"Needs Brain Region Map")
                slideSummaryData[year][pt['name']] = {"status": "incomplete", "reason": "needs brain region map"}
        
            else:

                ptItemData = mph.getFolderContents(gc,pt['_id']) ### Get all of the items cfor selected year
        
                stats = getPtSlideDataSummary(ptItemData,debug=True)
                slideSummaryData[year][pt['name']] = stats

In [4]:
needsMap = []
for yr in slideSummaryData:
    for pt in slideSummaryData[yr]:
        caseMeta = slideSummaryData[yr][pt]
        if caseMeta['status'] == 'parsed':
            print(yr,pt,caseMeta['valid'],caseMeta['notValid'])
        elif caseMeta['status'] == 'incomplete':
            needsMap.append((yr,pt))

2004 E04-152 70 19
2004 E04-169 62 3
2004 E04-186 73 1
2004 E04-49 71 0
2005 E05-04 74 3
2005 E05-130 67 0
2005 E05-132 69 0
2005 E05-179 66 0
2005 E05-194 61 34
2005 E05-56 74 0
2005 E05-74 40 0
2005 E05-81 92 0
2005 E05-87 52 0
2006 E06-06 61 92
2006 E06-128 69 0
2006 E06-13 67 0
2006 E06-15 69 0
2006 E06-155 73 0
2006 E06-18 54 53
2006 E06-41 68 0
2006 E06-97 62 54
2008 E08-101 88 4
2008 E08-145 46 0
2008 E08-162 138 25
2008 E08-53 62 6
2009 E09-08 72 40
2009 E09-136 86 3
2009 E09-155 55 88
2009 E09-164 236 37
2009 E09-81 115 22
2009 E09-91 184 46
2010 E10-110 59 64
2010 E10-129 84 54
2010 E10-137 55 62
2010 E10-150 79 49
2010 E10-151 85 39
2010 E10-160 285 25
2010 E10-179 86 46
2010 E10-18 0 74
2010 E10-33 102 7
2010 E10-38 89 51
2010 E10-48 77 62
2010 E10-56 86 46
2010 E10-63 78 42
2010 E10-64 0 77
2010 E10-81 180 40
2010 E10-88 78 11
2011 E11-112 78 46
2011 E11-124 106 16
2011 E11-125 109 63
2011 E11-128 98 9
2011 E11-139 76 54
2011 E11-143 96 31
2011 E11-20 86 15
2011 E11-41 98 

In [ ]:
mph.auditMetadata(gc,collectionID=NeuroPathDemoImages_Collection,outputRecords=True)

In [ ]:
# folder2020 = '638e81331f75016b81febf39'

# itemData2020 = mph.getFolderContents(gc,folder2020)

In [ ]:
allNpWorking = []

for itm in itemData2020:
    if 'meta' in itm and 'npWorking' in itm['meta']:
        allNpWorking.append(itm)

In [ ]:
[(x['meta']['npWorking'],x['name']) for x in allNpWorking]

In [ ]:
for pt in slideSummaryData['2017'].keys():
    s=  slideSummaryData['2017'][pt]
    if 'valid' in s:
        print('2017',pt,s['valid'],s['notValid'],s['controlSlides'])
    

In [ ]:
for i in slideSummaryData['2017']['E17-39']['slidesToReview']:
    print(i)

In [ ]:
allItemData = mph.get

In [ ]:
# for y in gc.listFolder(NeuroPathDemoImages_Collection,parentFolderType='collection'):
#     if y['name'].startswith('20'):
#         for ptId in gc.listFolder(y['_id']):
#             print(ptId['name'])
#             hlprs.scanMetadata(gc,ptId,updateGirder=True)

In [ ]:
# ## Now also flatten the views so in the image organizer app I don't have to upload

# for y in gc.listFolder(NeuroPathDemoImages_Collection,parentFolderType='collection'):
#     if y['name'].startswith('20'):
#         for ptId in gc.listFolder(y['_id']):
#             gc.addMetadataToFolder(ptId['_id'],{"isLinear":True, "caseViewFlag":1})

In [ ]:
# ### Get a list of every patient ID... this is quick, it's just from the main folder name!!
# ptIdList = []

# for yrf in gc.listFolder(NeuroPathDemoImages_Collection,parentFolderType='collection'):
#     year = yrf['name']
#     if year.isdigit():
#         for ptf in gc.listFolder(yrf['_id']):
# #             print("\t",ptf['name'])
#             ptIdList.append(ptf['name'])